### AutoGen Project

## Project: Creator

This is an answer to the idea: could an Agentic Framework create Agents itself?

In [1]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from agent import Agent
from creator import Creator
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_core import AgentId
import messages
import glob
import importlib
import os
from autogen_agentchat.messages import TextMessage


In [2]:

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [3]:
host

In [4]:
worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
await worker.start()

In [5]:

name = "Creator"
result = await Creator.register(worker, name, lambda: Creator(name))
print(result)
creator_id = AgentId(name, "default")

AgentType(type='Creator')


In [6]:
name = "agent1.py"
await worker.send_message(messages.Message(content="boo"), creator_id)

ValidationError: 1 validation error for TextMessage
source
  Field required [type=missing, input_value={'content': 'boo'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
async def create_agents():
    how_many_agents = 2
    new_agents = 0
    count = 1

    while new_agents < how_many_agents:
        name = f"agent{count}.py"
        if not os.path.exists(name):
            print(f"Creating agent {name}")
            await creator_worker.send_message(Message(content=name), creator_id)
            new_agents += 1
        count += 1

await create_agents()

In [ ]:
workers = []
agent_ids = []
count = 1

async def run_agents():

    agent_files = glob.glob("agent*.py")
    module_names = [os.path.splitext(file)[0] for file in agent_files]
        
    for name in module_names:
        try:
            module = importlib.import_module(name)
            worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
            await worker.start()
            await module.Agent.register(worker, name, lambda: module.Agent(name))
            workers.append(worker)
            agent_id = AgentId(name, "default")
            agent_ids.append(agent_id)
            response = await worker.send_message(Message(content="Please give me brief business idea in 2-3 sentences."), agent_id)
            print(response.content)
        except Exception as e:
            print(f"Error with this agent: {name}: {e}")

await run_agents()

In [ ]:
for worker in workers + [creator_worker]:
    try:
        await worker.stop()
    except Exception as e:
        print(f"Error stopping worker: {e}")
try:
    await host.stop()   
except Exception as e:
    print(f"Error stopping host: {e}")

In [ ]:
await worker.stop()
await host.stop()